In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

# タイトルと説明
st.title("データ解析アプリケーション")
st.write("""
    このアプリケーションは、指定されたデータに対して回帰直線の計算と波形の補正を行います。
    各種測定データの補正後の波形を表示し、平均値を計算します。また、FFT解析を行い、結果を表示します。
""")

# データのアップロード
uploaded_file = st.file_uploader("データファイルをアップロードしてください (CSV形式)", type="csv")

if uploaded_file is not None:
    # データの読み込み
    data_origin = pd.read_csv(uploaded_file, encoding="shift-jis", skiprows=61)

    # データの前処理
    data = data_origin.copy()
    data.drop(data.tail(39).index, inplace=True)
    data["日時(μs)"] = data["日時(μs)"].astype(str)
    data_join = data["#EndHeader"] + "." + data["日時(μs)"].str.rjust(6, '0')
    data["#EndHeader"] = pd.to_datetime(data_join)
    data.rename(columns={'日時(μs)': '日時(μs)', '(1)CA-CH01': '主軸方向抵抗', '(1)CA-CH02': '接線方向抵抗', '(1)CA-CH03': '法線方向抵抗', 
                         '(1)CA-CH04': '加速度センサ', '(2)HA-V01': '主軸モータ動力', '(2)HA-V02': '渦電流計', '(2)HA-V03': 'モータ電流', 
                         '(2)HA-V04': '熱電対', '(3)CA-CH01': 'マイクロフォン'}, inplace=True)
    R = ["主軸方向抵抗", "接線方向抵抗", "法線方向抵抗", "加速度センサ", "主軸モータ動力", "渦電流計", "モータ電流", "熱電対", "マイクロフォン"]
    data = data.set_index(data["#EndHeader"])
    data = data.iloc[:, 1:]
    data5 = data.astype(float)

    # データのコピーと日時補正
    data6 = data5.copy()
    data60 = np.array(data6["日時(μs)"])
    k = 0
    for i in range(len(data6)):
        if data60[i] == 0:
            k += 1
        data60[i] += 1000000 * k
    data60 = (data60 - data60[0]) * 0.000005
    data6["日時(μs)"] = data60

    # コメントデータの読み込みと前処理
    data2 = data_origin.copy()
    data2.drop(data2.tail(1).index, inplace=True)
    mark1 = data2[-37:]
    mark2 = mark1.copy()
    mark2.columns = mark1.iloc[0]
    mark3 = mark2.iloc[1:, :6]
    mark3["日時(結合)"] = pd.to_datetime(mark3["日時"] + "." + mark3["日時(μs)"])
    mark4 = mark3[["コメント", "日時(結合)"]]

    # グラフの設定
    plt.rcParams["figure.figsize"] = [25, 4.0]
    plt.rcParams['font.family'] = 'IPAexGothic'

    # データ表示
    st.subheader("生波形表示")
    fig, ax = plt.subplots(len(R), 1, figsize=(25, len(R) * 4))
    for i in range(len(R)):
        ax[i].plot((data[R[i]]))
        ax[i].set_title(R[i], fontsize=16)
    st.pyplot(fig)

    # 指定した波形の切り取りを表示
    wave_name = st.selectbox("波形名を選択してください", R)
    wave_data = data6[wave_name]

    # csvファイルでoriginが始まる位置
    origin = 20

    # 4つの配列を結合するためのリスト
    x_all = []
    y_all = []

    # 回帰直線の計算と波形表示
    for n in range(4):
        start = mark4.iloc[4 * n + origin, 1]
        end = mark4.iloc[4 * n + origin + 1, 1]
        
        x = np.array(data6.loc[start:end, '日時(μs)'])
        y = np.array(wave_data[start:end])

        # データをリストに追加
        x_all.extend(x)
        y_all.extend(y)

    # まとめた配列で傾きと切片を求める
    x_all = np.array(x_all)
    y_all = np.array(y_all)

    # 傾きと切片を求める
    a_all = ((x_all * y_all).mean() - (x_all.mean() * y_all.mean())) / ((x_all ** 2).mean() - x_all.mean() ** 2)
    b_all = -(a_all * x_all.mean()) + y_all.mean()

    st.write(f'まとめた配列の傾き={a_all}, 切片={b_all}')

    # まとめた配列で回帰直線を描画
    fig, ax = plt.subplots()
    ax.plot(x_all, y_all, label='Original')
    ax.plot(x_all, a_all * x_all + b_all, color='red', label='Regression Line')
    ax.set_title('Combined Regression Line')
    ax.set_xlabel('日時(μs)')
    ax.set_ylabel(wave_name)
    ax.legend()
    ax.grid(True)
    st.pyplot(fig)

    # 平均傾きと切片の計算
    avg_ab1 = (a_all, b_all)
    avg_ab2 = (a_all, b_all)

    # 波形の補正と表示
    st.subheader("補正後の波形表示")
    corrected_waveforms = []
    titles = []

    # 最初の5つの波形を補正
    for i in range(5):
        start = mark4.iloc[2 * i, 1]
        end = mark4.iloc[2 * i + 1, 1]
        x = np.array(data6.loc[start:end, '日時(μs)'])
        y = np.array(wave_data[start:end])
        
        corrected_y = y - (avg_ab2[0] * x + avg_ab2[1])
        corrected_waveforms.append(corrected_y)
        
        title = f'C_{"U" if i % 2 == 0 else "D"}_{i + 1}'
        titles.append(title)
        
        fig, ax = plt.subplots()
        ax.plot(x, y, label='Original')
        ax.plot(x, corrected_y, label='Corrected')
        ax.set_title(title)
        ax.set_xlabel('日時(μs)')
        ax.set_ylabel(wave_name)
        ax.grid(True)
        ax.legend()
        st.pyplot(fig)

    # 後ろの5つの波形を補正
    for i in range(5, 10):
        start = mark4.iloc[2 * i, 1]
        end = mark4.iloc[2 * i + 1, 1]
        x = np.array(data6.loc[start:end, '日時(μs)'])
        y = np.array(wave_data[start:end])
        
        corrected_y = y - (avg_ab1[0] * x + avg_ab1[1])
        corrected_waveforms.append(corrected_y)
        
        title = f'F_{"U" if (i - 5) % 2 == 0 else "D"}_{i - 4}'
        titles.append(title)
        
        fig, ax = plt.subplots()
        ax.plot(x, y, label='Original')
        ax.plot(x, corrected_y, label='Corrected')
        ax.set_title(title)
        ax.set_xlabel('日時(μs)')
        ax.set_ylabel(wave_name)
        ax.grid(True)
        ax.legend()
        st.pyplot(fig)

    # 各補正波形の平均値を計算して表示
    st.subheader("補正波形の平均値")
    for i, (title, corrected_y) in enumerate(zip(titles, corrected_waveforms)):
        st.write(f'{title} Mean: {np.mean(corrected_y)}')

    # 各抵抗データに対して処理を行う
    data7 = data6.copy()
    for resistance in R:
        x_all = []
        y_all = []

        for n in range(4):
            start = mark4.iloc[4 * n + origin, 1]
            end = mark4.iloc[4 * n + origin + 1, 1]
            
            x = np.array(data6.loc[start:end, '日時(μs)'])
            y = np.array(data6.loc[start:end, resistance])
            
            x_all.extend(x)
            y_all.extend(y)

        # まとめた配列で傾きと切片を求める
        x_all = np.array(x_all)
        y_all = np.array(y_all)

        # 傾きと切片を求める
        a_all = ((x_all * y_all).mean() - (x_all.mean() * y_all.mean())) / ((x_all ** 2).mean() - x_all.mean() ** 2)
        b_all = -(a_all * x_all.mean()) + y_all.mean()

        st.write(f'{resistance}のまとめた配列の傾き={a_all}, 切片={b_all}')

        avg_ab1 = (a_all, b_all)
        avg_ab2 = (a_all, b_all)

        # 波形の補正と表示
        corrected_waveforms = []
        titles = []

        # 波形を補正
        x = data6["日時(μs)"]
        y = data6[resistance]
               
        corrected_y = y - (avg_ab1[0] * x + avg_ab1[1])
        corrected_waveforms.append(corrected_y)
        
        data7[resistance] = corrected_y
        
        fig, ax = plt.subplots()
        ax.plot(data6["日時(μs)"], data6[resistance], label='Original')
        ax.plot(x, corrected_y, label='Corrected')
        ax.set_xlabel('日時(μs)')
        ax.set_ylabel(resistance)
        ax.grid(True)
        ax.legend()
        st.pyplot(fig)

    # 全測定値を表示
    for resistance in R:
        x_all = []
        y_all = []

        for n in range(4):
            start = mark4.iloc[4 * n + origin, 1]
            end = mark4.iloc[4 * n + origin + 1, 1]
            
            x = np.array(data6.loc[start:end, '日時(μs)'])
            y = np.array(data6.loc[start:end, resistance])
            
            x_all.extend(x)
            y_all.extend(y)

        x_all = np.array(x_all)
        y_all = np.array(y_all)

        a_all = ((x_all * y_all).mean() - (x_all.mean() * y_all.mean())) / ((x_all ** 2).mean() - x_all.mean() ** 2)
        b_all = -(a_all * x_all.mean()) + y_all.mean()

        st.write(f'{resistance}のまとめた配列の傾き={a_all}, 切片={b_all}')

        avg_ab1 = (a_all, b_all)
        avg_ab2 = (a_all, b_all)

        corrected_waveforms = []
        titles = []

        for i in range(5):
            start = mark4.iloc[2 * i, 1]
            end = mark4.iloc[2 * i + 1, 1]
            x = np.array(data6.loc[start:end, '日時(μs)'])
            y = np.array(data6.loc[start:end, resistance])
            
            corrected_y = y - (avg_ab2[0] * x + avg_ab2[1])
            corrected_waveforms.append(corrected_y)
            
            title = f'C_{"U" if i % 2 == 0 else "D"}_{i + 1}'
            titles.append(title)

        for i in range(5, 10):
            start = mark4.iloc[2 * i, 1]
            end = mark4.iloc[2 * i + 1, 1]
            x = np.array(data6.loc[start:end, '日時(μs)'])
            y = np.array(data6.loc[start:end, resistance])
            
            corrected_y = y - (avg_ab1[0] * x + avg_ab1[1])
            corrected_waveforms.append(corrected_y)
            
            title = f'F_{"U" if (i - 5) % 2 == 0 else "D"}_{i - 4}'
            titles.append(title)

        for i, (title, corrected_y) in enumerate(zip(titles, corrected_waveforms)):
            st.write(f'{title} Mean: {np.mean(corrected_y)}')

    # FFTを実行して結果をプロットする関数
    def perform_fft(data7, start, end, sample_rate=20000):
        # 加速度データの該当部分を抽出
        acc_data = np.array(data7.loc[start:end, '加速度センサ'])
        
        # FFTを実行
        N = len(acc_data)
        T = 1 / sample_rate
        yf = fft(acc_data)
        xf = fftfreq(N, T)[:N//2]
        
        # FFTの振幅を計算
        amplitude = 2.0 / N * np.abs(yf[0:N//2])
        
        # ピーク値を見つける
        peak_indices = np.argsort(amplitude)[-5:][::-1]  # 上位5つのピークのインデックス
        peak_freqs = xf[peak_indices]  # 上位5つのピークの周波数
        peak_amps = amplitude[peak_indices]  # 上位5つのピークの振幅
        
        # FFT結果をプロット
        plt.figure(figsize=(25, 4.0))
        plt.plot(xf, amplitude)
        plt.title(f'{start} から {end} までの加速度データのFFT')
        plt.xlabel('周波数 (Hz)')
        plt.ylabel('振幅')
        plt.grid(True)
        
        # ピーク値を注釈
        for freq, amp in zip(peak_freqs, peak_amps):
            plt.annotate(f'{freq:.1f} Hz', xy=(freq, amp), xytext=(freq + 100, amp),
                         arrowprops=dict(facecolor='red', shrink=0.05))
        
        plt.show()
        
        return xf, amplitude

    # 指定した周波数の加速度値を取得する関数
    def get_acceleration_at_frequency(freq, xf, amplitude):
        # 最も近い周波数のインデックスを見つける
        idx = (np.abs(xf - freq)).argmin()
        return amplitude[idx]

    # FFT解析のための区間を指定
    intervals = [(mark4.iloc[2*i, 1], mark4.iloc[2*i+1, 1]) for i in range(10)]

    # 各区間に対してFFTを実行し、結果をプロット
    fft_results = []
    for start, end in intervals:
        xf, amplitude = perform_fft(data7, start, end)
        fft_results.append((xf, amplitude))

    for i in range(5):
        xf, amplitude = fft_results[i]  #オリジンを除くマーク付けした区間(fft_resultsの区間数以下でないと動かない)
        peak_indices = np.argsort(amplitude)[-5:][::-1]  # 上位5つのピークのインデックス
        peak_freqs = xf[peak_indices]  # 上位5つのピークの周波数
        peak_amps = amplitude[peak_indices]  # 上位5つのピークの振幅
        title = f'C_{"U" if i % 2 == 0 else "D"}_{i+1}'
        st.write(title)
        for n in range(5):
            freq = peak_freqs[n]  # 上位5つのピークの振幅の加速度を抽出
            acc_value = get_acceleration_at_frequency(freq, xf, amplitude)
            st.write(f'{freq} Hz での加速度値: {acc_value/1000}')  # Vに変換

    for i in range(5, 10):
        xf, amplitude = fft_results[i]  
        peak_indices = np.argsort(amplitude)[-5:][::-1]  
        peak_freqs = xf[peak_indices]  
        peak_amps = amplitude[peak_indices]  
        title = f'F_{"U" if (i-5) % 2 == 0 else "D"}_{i-4}'
        st.write(title)
        for n in range(5):
            freq = peak_freqs[n] 
            acc_value = get_acceleration_at_frequency(freq, xf, amplitude)
            st.write(f'{freq} Hz での加速度値: {acc_value/1000}') 

    freq = st.slider('frequency', 0.0, 100000, 1.0)  # ピックアップしたい加速度を入力
    acc_value = get_acceleration_at_frequency(freq, xf, amplitude)

    for i in range(5):
        xf, amplitude = fft_results[i]  # オリジンを除くマーク付けした区間(fft_resultsの区間数以下でないと動かない)
        title = f'C_{"U" if i % 2 == 0 else "D"}_{i+1}'
        st.write(title)
        st.write(f'{freq} Hz での加速度値: {acc_value}') 

    for i in range(5, 10):
        xf, amplitude = fft_results[i]  
        title = f'F_{"U" if (i-5) % 2 == 0 else "D"}_{i-4}'
        st.write(title)
        st.write(f'{freq} Hz での加速度値: {acc_value}') 


2024-06-07 14:43:52.587 
  command:

    streamlit run c:\Users\rokun\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
